<a href="https://colab.research.google.com/github/balandongiv/mmocr_tutorial/blob/main/DEBUG_UNABLE_TO_RESUME_TRAINING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DEBUG: UNABLE TO RESUME TRAINING



# Install MMOCR

### Install Dependencies 

In [ ]:
%cd ..
# Install mmcv-full thus we could use CUDA operators
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu113/torch1.11.0/index.html

# Install mmdetection
!pip install mmdet

# # Install mmocr
!git clone https://github.com/open-mmlab/mmocr.git
%cd mmocr
!pip install -r requirements.txt
!pip install -v -e .

### Check Installed Dependencies Versions

In [2]:
# Check Pytorch installation
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check MMDetection installation
import mmdet
print(mmdet.__version__)

# Check mmcv installation
import mmcv
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(mmcv.__version__)
print(get_compiling_cuda_version())
print(get_compiler_version())

# Check mmocr installation
import mmocr
print(mmocr.__version__)

%cd /mmocr/
!ls

1.11.0+cu113 True
2.25.0
1.5.3
11.3
GCC 7.3
0.6.0
/mmocr
CITATION.cff  docs	   mmocr.egg-info   requirements      setup.py
configs       LICENSE	   model-index.yml  requirements.txt  tests
demo	      MANIFEST.in  README.md	    resources	      tools
docker	      mmocr	   README_zh-CN.md  setup.cfg


Lets download a `zip` which contain


1.   Pretrain crnn model which has been train with `Syn90k` for 6 epochs with the name `epoch_6.pth`
2.   List of images with its corresponding coco annotation stored under the folder `imgs`



In [ ]:
""
# Download the zip file

!gdown --id 1fKnXVj0Xf8SJmdvoSWrIlW2kxECC7r-8 --output /content/mmocr_debug.zip

# Unzip the file
!unzip /content/mmocr_debug.zip -d /content



In [26]:

# Convert from COCO to jsonl
# train_img_prefix ='/content/mmocr_debug/val/mmocr_compatible_annotation/crops'
# train_ann_filex = '/content/mmocr_debug/val/mmocr_compatible_annotation/train_label.jsonl'
!python /mmocr/tools/data/common/labelme_converter.py /content/mmocr_debug/val /content/mmocr_debug/val /content/mmocr_debug/val/mmocr_compatible_annotation --tasks recog



# Convert from COCO to jsonl
# train_img_prefix ='/content/mmocr_debug/train/mmocr_compatible_annotation/crops'
# train_ann_filex = '/content/mmocr_debug/train/mmocr_compatible_annotation/train_label.jsonl'

!python /mmocr/tools/data/common/labelme_converter.py /content/mmocr_debug/train /content/mmocr_debug/train /content/mmocr_debug/train/mmocr_compatible_annotation --tasks recog

[                                                  ] 0/21, elapsed: 0s, ETA:libpng warning: bKGD: invalid
[  ] 1/21, 8.9 task/s, elapsed: 0s, ETA:     2slibpng warning: bKGD: invalid
[  ] 2/21, 9.3 task/s, elapsed: 0s, ETA:     2slibpng warning: bKGD: invalid
[  ] 3/21, 9.7 task/s, elapsed: 0s, ETA:     2slibpng warning: bKGD: invalid
[  ] 4/21, 9.5 task/s, elapsed: 0s, ETA:     2slibpng warning: bKGD: invalid
[  ] 5/21, 9.5 task/s, elapsed: 1s, ETA:     2slibpng warning: bKGD: invalid
[  ] 6/21, 9.6 task/s, elapsed: 1s, ETA:     2slibpng warning: bKGD: invalid
[  ] 7/21, 9.6 task/s, elapsed: 1s, ETA:     1slibpng warning: bKGD: invalid
[  ] 8/21, 9.6 task/s, elapsed: 1s, ETA:     1slibpng warning: bKGD: invalid
[  ] 9/21, 9.7 task/s, elapsed: 1s, ETA:     1slibpng warning: bKGD: invalid
[  ] 10/21, 9.7 task/s, elapsed: 1s, ETA:     1slibpng warning: bKGD: invalid
[> ] 11/21, 9.7 task/s, elapsed: 1s, ETA:     1slibpng warning: bKGD: invalid
[> ] 12/21, 9.7 task/s, elapsed: 1s, ETA:    

## Resume training from the `epoch_6.pth` with a new list of crop `images`




In [27]:
import os.path as osp

import mmcv
from mmcv import Config
from mmdet.apis import set_random_seed

from mmocr.apis import train_detector
from mmocr.datasets import build_dataset
from mmocr.models import build_detector


cfg = Config.fromfile('/content/mmocr_debug/cfg_debug_resume.py')

seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)

# Let also limit the training to only 14 epochs

cfg.runner = dict(type='EpochBasedRunner', max_epochs=14) # Default max_epochs is 1200

#save the model every 2 epoch
cfg.checkpoint_config = dict(interval=2) # Default the interval was set to 100

print(f'Config:\n{cfg.pretty_text}')

# Build dataset
datasets = [build_dataset(cfg.data.train)]
#
# # Build the detector
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
#
# # Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# # Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
2022-06-24 10:26:42,051 - mmocr - INFO - load checkpoint from local path: /content/mmocr_debug/epoch_6.pth
2022-06-24 10:26:42,146 - mmocr - INFO - resumed epoch 6, iter 836184
2022-06-24 10:26:42,148 - mmocr - INFO - Start running, host: root@7b1fd3e97bc2, work_dir: /content/mmocr_debug/resume
2022-06-24 10:26:42,151 - mmocr - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(NORMAL      ) EvalHook                       

Config:
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = '/content/mmocr_debug/epoch_6.pth'
workflow = [('train', 1)]
opencv_num_threads = 0
mp_start_method = 'fork'
label_convertor = dict(
    type='CTCConvertor', dict_type='DICT36', with_unknown=False, lower=True)
model = dict(
    type='CRNNNet',
    preprocessor=None,
    backbone=dict(type='VeryDeepVgg', leaky_relu=False, input_channels=1),
    encoder=None,
    decoder=dict(type='CRNNDecoder', in_channels=512, rnn_flag=True),
    loss=dict(type='CTCLoss'),
    label_convertor=dict(
        type='CTCConvertor',
        dict_type='DICT36',
        with_unknown=False,
        lower=True),
    pretrained=None)
img_norm_cfg = dict(mean=[127], std=[127])
train_pipeline = [
    dict(type='LoadImageFromFile', color_type='grayscale'),
    dict(
        type='ResizeOCR',
        height=32,
        min_width=100,
        max_width=100,
   

2022-06-24 10:26:45,412 - mmocr - INFO - 
Evaluating /content/mmocr_debug/val/mmocr_compatible_annotation/train_label.jsonl with 63 images now
2022-06-24 10:26:45,418 - mmocr - INFO - Epoch(val) [7][63]	0_char_recall: 0.7937, 0_char_precision: 0.8130, 0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.6190, 0_word_acc_ignore_case_symbol: 0.6190, 0_1-N.E.D: 0.7937
2022-06-24 10:26:47,782 - mmocr - INFO - Saving checkpoint at 8 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 63/63, 81.5 task/s, elapsed: 1s, ETA:     0s

2022-06-24 10:26:49,073 - mmocr - INFO - 
Evaluating /content/mmocr_debug/val/mmocr_compatible_annotation/train_label.jsonl with 63 images now
2022-06-24 10:26:49,076 - mmocr - INFO - Epoch(val) [8][63]	0_char_recall: 0.8175, 0_char_precision: 0.8175, 0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.6349, 0_word_acc_ignore_case_symbol: 0.6349, 0_1-N.E.D: 0.8122


...

2022-06-24 10:27:02,975 - mmocr - INFO - 
Evaluating /content/mmocr_debug/val/mmocr_compatible_annotation/train_label.jsonl with 63 images now
2022-06-24 10:27:02,979 - mmocr - INFO - Epoch(val) [12][63]	0_char_recall: 0.8889, 0_char_precision: 0.8889, 0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.7937, 0_word_acc_ignore_case_symbol: 0.7937, 0_1-N.E.D: 0.8889


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 63/63, 81.3 task/s, elapsed: 1s, ETA:     0s

2022-06-24 10:27:06,229 - mmocr - INFO - 
Evaluating /content/mmocr_debug/val/mmocr_compatible_annotation/train_label.jsonl with 63 images now
2022-06-24 10:27:06,232 - mmocr - INFO - Epoch(val) [13][63]	0_char_recall: 0.8889, 0_char_precision: 0.8889, 0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.7937, 0_word_acc_ignore_case_symbol: 0.7937, 0_1-N.E.D: 0.8889
2022-06-24 10:27:08,587 - mmocr - INFO - Saving checkpoint at 14 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 63/63, 76.8 task/s, elapsed: 1s, ETA:     0s

2022-06-24 10:27:09,910 - mmocr - INFO - 
Evaluating /content/mmocr_debug/val/mmocr_compatible_annotation/train_label.jsonl with 63 images now
2022-06-24 10:27:09,915 - mmocr - INFO - Epoch(val) [14][63]	0_char_recall: 0.8968, 0_char_precision: 0.8968, 0_word_acc: 0.0000, 0_word_acc_ignore_case: 0.8095, 0_word_acc_ignore_case_symbol: 0.8095, 0_1-N.E.D: 0.8968
